In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp import *
from shared_utils import utils

# Formatting the nb
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# Specific to this project
import A1_utilities as utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/pmp_dashboard/"

In [3]:
unwanted_appropriations = []

In [4]:
ap11 = utils.import_raw_data(
    "FY 2122 AP11_Closed_PMP Summary Report.022822_Updated.xlsx",
    "Raw Data AP11 Closed",
    unwanted_appropriations,
)

In [5]:
ap10 = utils.import_raw_data(
    "FY 2122 AP10_Closed_PMP Summary Report.022822_Andrew Updated.xlsx",
    "Raw Data AP10 Closed",
    unwanted_appropriations,
)

In [12]:
ap12 = utils.import_raw_data(
    "FY 2122 AP12_Closed_PMP Summary Report.022822_Updated.xlsx",
    "Raw Data AP12 Closed",
    unwanted_appropriations,
)

In [6]:
def pmp_dashboard_sheets(df):
    
    fund_by_div = utils.create_fund_by_division(df)
    tspoe = utils.create_tpsoe(df, utils.tpsoe_ps_list, utils.tpsoe_oe_list)
    timeline = utils.create_timeline(utils.my_clean_dataframes)
    psoe = utils.create_psoe_timeline(timeline, utils.psoe_ps_cols, utils.psoe_oe_cols)

    # How to save this to sharepoint
    return fund_by_div, tspoe, timeline, psoe

In [7]:
df1, df2, df3, df4 = pmp_dashboard_sheets(utils.import_raw_data(
    "FY 2122 AP12_Closed_PMP Summary Report.022822_Updated.xlsx",
    "Raw Data AP12 Closed",
    unwanted_appropriations,
))

/home/jovyan/data-analyses/pmp_dashboard/A1_utilities.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/pmp_dashboard/A1_utilities.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/pmp_dashboard/A1_utilities.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [11]:
len(df1)

55

In [9]:
df1_cols = ['ps_allocation', 'ps_expenditure',
       'ps_balance', 'ps_projection', 
        'oe_allocation', 'oe_encumbrance', 'oe_expenditure',
       'oe_balance', 'oe_projection', 'total_allocation',
       'total_expenditure', 'total_balance', 'total_projection',]

In [10]:
for i in df1_cols:
    print("\n" + i)
    print(df1[i].sum())


ps_allocation
244928000

ps_expenditure
211313061.69999996

ps_balance
33614938.3

ps_projection
211313061.69999996

oe_allocation
233766000

oe_encumbrance
249639535.57000005

oe_expenditure
97354512.35999998

oe_balance
-113228047.92999998

oe_projection
346949724.18801653

total_allocation
478694000

total_expenditure
558307109.6300001

total_balance
-79613109.63000003

total_projection
558262785.8880167


In [21]:
df2_cols = ['allocation', 'expenditure', 'balance', 'encumbrance',
       'projection']

In [22]:
for i in df2_cols:
    print("\n" + i)
    print(df2[i].sum())


allocation
478694000.0

expenditure
308667574.06

balance
-79613109.63

encumbrance
249639535.57

projection
558262785.8880165


In [24]:
len(df3)

172

In [30]:
df3_cols = ['ps_allocation', 'ps_expenditure',
       'ps_balance', 'ps_projection', 'oe_allocation',
       'oe_encumbrance', 'oe_expenditure', 'oe_balance', 'oe_projection']